In [4]:
import urllib
import cv2
import numpy as np
import time
import warnings
import datetime
import imutils
import json

# Replace the URL with your own IPwebcam shot.jpg IP:port
url='http://192.168.43.1:8080//shot.jpg'
avg = None
#give the current time and store it in variable called lastuploaded
lastUploaded = datetime.datetime.now()
motionCounter = 0
#flag to check if first frame has been encountered
FirstFrame = None

text="Unoccupied"
timestamp = datetime.datetime.now()
ts=timestamp
var=1
while True:
    # Use urllib to get the image from the IP camera
    imgResp = urllib.request.urlopen(url)
    var=1
    # Numpy to convert into a array
    imgNp = np.array(bytearray(imgResp.read()),dtype=np.uint8)
    #print(imgNp)
    
    # Finally decode the array to OpenCV usable format ;) 
    img = cv2.imdecode(imgNp,-1)
    #Resizing the image
    frame = imutils.resize(img, width = 500)
    
    #convert image to grayscale
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #blurring to smoothen the image
    gray  = cv2.GaussianBlur(gray, (21,21), 0) #average pixel intensities, no 2 frames are 100 % same, that could throw false results
            
    #if the first frame is None, initialize it
    if FirstFrame is None:
        FirstFrame = gray
        continue

    #compute the absolute difference between the current frame and first frame
    frameDelta = cv2.absdiff(FirstFrame, gray)
    #print(frameDelta)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1] #highlight significant pixel changes

    #dilate the thresholded image to fill in holes, then find contours on thresholded image
    thresh = cv2.dilate(thresh, None, iterations = 2)
    
    #making rectangular bounding boxes
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    cnts = cnts[0] if imutils.is_cv2() else cnts[1] 
    #the above 2 lines is for contour detection to find outlines of differences

    #loop over the contours
    for c in cnts:
        #if contour is too small, ignore it
        if cv2.contourArea(c) > 500:
            #compute the bounding box for the contour, draw it on the frame and update the text
            (x, y, w, h) = cv2.boundingRect(c)
            #cv2.rectangle(image on which to draw, starting point, ending point, color in RGB, thickness in pixes)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            var=0
            #ts = timestamp.strftime("%A %d %B %Y %I:%M:%S%p")
            text = "Occupied"
            
    if var==1:
        text="Unoccupied"
    #draw the text and timestamp on the frame
   
    cv2.putText(frame, "Room Status : {}".format(text), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
   

    #show the frame and record if user presses a key
    cv2.imshow("Security Feed", frame)
    
    key = cv2.waitKey(1) & 0xFF

    #if 'q' is pressed, break from the loop
    if key == ord("q"):
        break
    
    
